In [1]:
import tensorflow as tf
import numpy as np
import sys

from sklearn.metrics import mean_squared_error
from keras.callbacks import LearningRateScheduler
#from sklearn.preprocessing import MinMaxScaler ##########

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
#sys.path.append('/Users/Think/Desktop/project')

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_Oscillator import OscillatorDataHandler, LinearPDEDataHandler

#Custom modules
from ann_framework import aux_functions

import aux_functions_stochastic
import analytic_functions
import loss_functions


Using TensorFlow backend.


In [2]:
#global variables

#deltas = [0.1, 0.1]
deltas = [0.01]

k = 1
c = 0.1
D = 1
num_fevals = 5

sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)

### Define tensorflow model

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), 
                         name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y


def tf_ghazale(X):
    
    l2_lambda_regularization = 0.20
    #l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 10, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_lambda_regularization), 
                         name="fc1")
    y = tf.layers.dense(A1, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_lambda_regularization), name="out")
    
    return y



def tf_model_yulin(X):
    
    A1 = tf.layers.dense(X, 32, activation=tf.nn.tanh, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),name="fc1")
    """
    dropout1 = tf.layers.dropout(inputs = A1, rate = 0.2)
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    dropout2 = tf.layers.dropout(inputs = A2, rate = 0.2)
    A3 = tf.layers.dense(A2, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc3")
    dropout3 = tf.layers.dropout(inputs = A3, rate = 0.2)
    A4 = tf.layers.dense(A3, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc4")
    dropout4 = tf.layers.dropout(inputs = A4, rate = 0.2)
    """
    y = tf.layers.dense(A1, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        name="out")
    #activation funcs
    
    return y


### Create Model TF

In [8]:
def tf_compiled_model(num_features, output_shape, num_fevals=1):
    tf.reset_default_graph()

    X, y = create_placeholders(num_features, output_shape)

    #y_pred = tf_model(X)
    #y_pred = tf_model_yulin(X)
    y_pred = tf_ghazale(X)
    
    #loss_function = loss_functions.squared_residual_function_wrapper2(k, c, D, deltas, num_fevals)
    loss_function =  loss_functions.linear_residual_function_wrapper(num_features, output_shape, deltas, num_fevals)
    cost, e = loss_function(X, y_pred, y)
    #reg_cost = tf.losses.get_regularization_loss()
    total_cost = e
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.5).minimize(total_cost)

    return {'X_placeholder': X, 'y_placeholder': y, 'y_pred': y_pred, 'cost': cost, 'total_cost': total_cost, 'optimizer': optimizer}

### Create Tunable Model Tensorflow and assign data

In [ ]:
"""

#scaler = MinMaxScaler()

dhandler_stochastic = OscillatorDataHandler()

#(data_scaler = scaler)

model = tf_compiled_model(2, 1, 5)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[10, 10], n=[50, 50])

#Real phi function
p_real_test, phi_real_test = analytic_functions.real_p(tModel.X_test[:,0], tModel.X_test[:,1], sigma_x, sigma_y)
p_real_train, phi_real_train = analytic_functions.real_p(tModel.X_train[:,0], tModel.X_train[:,1], sigma_x, sigma_y)
p_real_crossVal, phi_real_crossVal = analytic_functions.real_p(tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], sigma_x, sigma_y)

tModel.y_test = phi_real_test
tModel.y_train = phi_real_train
tModel.y_crossVal = phi_real_crossVal


tModel.print_data()
"""

## For linear PDE

In [9]:
#scaler = MinMaxScaler()

dhandler_stochastic = LinearPDEDataHandler()

#(data_scaler = scaler)

model = tf_compiled_model(1, 1, 2)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic, batch_size=8)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=0, boundaries=10, n=100)
#Real phi function
tModel.y_test = analytic_functions.real_y(tModel.X_test[:,0])
tModel.y_train = analytic_functions.real_y(tModel.X_train[:,0])
tModel.y_crossVal = analytic_functions.real_y(tModel.X_crossVal[:,0])

tModel.print_data()

Loading data for the first time
Reloading data due to parameter change
Loading data. Cros-Validation ratio 0.2
[[-10.        ]
 [ -9.89949749]
 [ -9.79899497]
 [ -9.69849246]
 [ -9.59798995]
 [ -9.49748744]
 [ -9.39698492]
 [ -9.29648241]
 [ -9.1959799 ]
 [ -9.09547739]
 [ -8.99497487]
 [ -8.89447236]
 [ -8.79396985]
 [ -8.69346734]
 [ -8.59296482]
 [ -8.49246231]
 [ -8.3919598 ]
 [ -8.29145729]
 [ -8.19095477]
 [ -8.09045226]
 [ -7.98994975]
 [ -7.88944724]
 [ -7.78894472]
 [ -7.68844221]
 [ -7.5879397 ]
 [ -7.48743719]
 [ -7.38693467]
 [ -7.28643216]
 [ -7.18592965]
 [ -7.08542714]
 [ -6.98492462]
 [ -6.88442211]
 [ -6.7839196 ]
 [ -6.68341709]
 [ -6.58291457]
 [ -6.48241206]
 [ -6.38190955]
 [ -6.28140704]
 [ -6.18090452]
 [ -6.08040201]
 [ -5.9798995 ]
 [ -5.87939698]
 [ -5.77889447]
 [ -5.67839196]
 [ -5.57788945]
 [ -5.47738693]
 [ -5.37688442]
 [ -5.27638191]
 [ -5.1758794 ]
 [ -5.07537688]
 [ -4.97487437]
 [ -4.87437186]
 [ -4.77386935]
 [ -4.67336683]
 [ -4.57286432]
 [ -4.472

/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Train model Tensorflow

In [10]:
tModel.epochs = 200
#lrate = LearningRateScheduler(aux_functions.step_decay)
minibatches_function_handle = aux_functions_stochastic.get_minibatches_linear

#tf.get_variable("deltas", initializer=deltas)

sess = tf.Session()

#writer = tf.summary.FileWriter('./graphs', sess.graph)

tModel.train_model(tf_session=sess, get_minibatches_function_handle=minibatches_function_handle, 
                   verbose=1, deltas=deltas)

    # cost_reg: total_cost -> e
    # cost: R

Epoch: 0001 cost_reg= 6858598.367540147 cost= 4.216958778
Epoch: 0002 cost_reg= 8533741.057169013 cost= 0.209280640
Epoch: 0003 cost_reg= 8246784.682675255 cost= 2.500413544
Epoch: 0004 cost_reg= 6262631.911755030 cost= 7.065452405
Epoch: 0005 cost_reg= 5927725.393853432 cost= 29.706732286
Epoch: 0006 cost_reg= 6073941.956219143 cost= 65.752970219
Epoch: 0007 cost_reg= 6647566.470823500 cost= 141.506452084
Epoch: 0008 cost_reg= 6921232.771535237 cost= 294.450199127
Epoch: 0009 cost_reg= 7370475.617519114 cost= 500.783875836
Epoch: 0010 cost_reg= 6913699.879801220 cost= 793.333850437
Epoch: 0011 cost_reg= 8219397.704352909 cost= 1520.697697957
Epoch: 0012 cost_reg= 6826260.368136936 cost= 2281.137571547
Epoch: 0013 cost_reg= 7178641.217783610 cost= 3239.562399970
Epoch: 0014 cost_reg= 8308508.073862712 cost= 4928.501586914
Epoch: 0015 cost_reg= 5931709.605916341 cost= 5989.411729601
Epoch: 0016 cost_reg= 7057392.156699287 cost= 8618.711686876
Epoch: 0017 cost_reg= 6477797.157670762 cost

Epoch: 0140 cost_reg= 3933457.566840278 cost= 4055191.531250000
Epoch: 0141 cost_reg= 3097188.156901042 cost= 3713086.263888889
Epoch: 0142 cost_reg= 3611082.840820312 cost= 3078709.666666667
Epoch: 0143 cost_reg= 3178164.157118056 cost= 3336217.239583333
Epoch: 0144 cost_reg= 3840313.026041667 cost= 3975627.805555556
Epoch: 0145 cost_reg= 3033328.677083333 cost= 3818117.229166667
Epoch: 0146 cost_reg= 3880787.774305556 cost= 4167511.250000000
Epoch: 0147 cost_reg= 2715357.090711806 cost= 3597742.545138889
Epoch: 0148 cost_reg= 3941606.212239583 cost= 3624471.709201389
Epoch: 0149 cost_reg= 4181870.645833333 cost= 3900369.621527778
Epoch: 0150 cost_reg= 4173870.064236111 cost= 4214865.638888889
Epoch: 0151 cost_reg= 2808734.947916667 cost= 3995452.723307292
Epoch: 0152 cost_reg= 3576558.699652778 cost= 3980991.538194444
Epoch: 0153 cost_reg= 3983500.086805556 cost= 3864307.691406250
Epoch: 0154 cost_reg= 3825659.337022569 cost= 3390390.449218750
Epoch: 0155 cost_reg= 3772344.773437500 

In [7]:
display_points = 20

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
X_test = tModel.X_crossVal
y_pred = tModel.y_predicted
y_real = tModel.y_crossVal
print("scores")

print(X_test)
print(y_real)

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

total_points = len(y_pred)
sample_array = list(range(total_points))

sample_points = np.random.choice(sample_array, display_points)
print(sample_points)

y_real_sampled = y_real[sample_points]
y_pred_sampled = y_pred[sample_points]
X_sampled = X_test[sample_points,:]

print(y_real_sampled)

i = range(len(y_pred_sampled))


for x, y_real_display, y_pred_display in zip(X_sampled, y_real_sampled, y_pred_sampled):
    print('x {}, Real y {}, Predicted y {}'.format(x, y_real_display, y_pred_display))

#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

scores
[[-4.27135678]
 [ 8.3919598 ]
 [-7.78894472]
 [ 3.96984925]
 [ 8.09045226]
 [ 3.5678392 ]
 [ 6.58291457]
 [ 2.96482412]
 [-7.28643216]
 [ 2.36180905]
 [-5.57788945]
 [-7.18592965]
 [-9.69849246]
 [ 1.35678392]
 [ 0.45226131]
 [-1.75879397]
 [ 1.45728643]
 [-4.07035176]
 [-3.26633166]
 [-4.3718593 ]
 [-8.69346734]
 [ 9.09547739]
 [ 8.19095477]
 [ 7.68844221]
 [ 4.87437186]
 [-4.77386935]
 [-9.1959799 ]
 [-2.36180905]
 [-0.25125628]
 [ 2.66331658]
 [ 8.69346734]
 [-2.06030151]
 [-1.65829146]
 [ 3.36683417]
 [ 5.57788945]
 [-2.16080402]]
[[1.39628258e-02]
 [4.41145479e+03]
 [4.14289844e-04]
 [5.29765438e+01]
 [3.26316304e+03]
 [3.54399316e+01]
 [7.22642457e+02]
 [1.93912924e+01]
 [6.84766838e-04]
 [1.06101283e+01]
 [3.78053613e-03]
 [7.57164784e-04]
 [6.13759519e-05]
 [3.88368296e+00]
 [1.57186263e+00]
 [1.72252480e-01]
 [4.29429085e+00]
 [1.70713824e-02]
 [3.81461037e-02]
 [1.26277400e-02]
 [1.67677622e-04]
 [8.91488283e+03]
 [3.60816558e+03]
 [2.18297130e+03]
 [1.30891909e+02]
 [

In [ ]:
display_points = 50

"""
#Evaluate real model
X_test = tModel.X_crossVal

#Real phi function
sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_test[:,0], X_test[:,1], sigma_x, sigma_y)

tModel.y_crossVal = phi_real

#p_real = np.ravel(p_real)
#phi_real = np.ravel(phi_real)

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
#phi_pred = np.ravel(tModel.y_predicted)
phi_pred = tModel.y_predicted

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)
"""

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
X_test = tModel.X_crossVal
phi_pred = tModel.y_predicted
phi_real = tModel.y_crossVal
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

total_points = len(phi_pred)
sample_array = list(range(total_points))

sample_points = np.random.choice(sample_array, display_points)

phi_real_sampled = phi_real[sample_points]
phi_pred_sampled = phi_pred[sample_points]
X_sampled = X_test[sample_points,:]

i = range(len(phi_pred_sampled))


for i, phi_real_display, phi_pred_display in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_test[i], phi_real_display, phi_pred_display))

#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig1 = plt.figure(1)
ax1 = Axes3D(fig1)
pred_phi_plot = Axes3D.scatter(ax1, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_pred)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('phi(x, y)_pred')

plt.show()

In [ ]:
fig2 = plt.figure(2)
ax2 = Axes3D(fig2)
real_phi_plot = Axes3D.scatter(ax2, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_real)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('phi(x, y)_real')

plt.show()

In [ ]:
fig3 = plt.figure(3)
ax3 = Axes3D(fig3)
pred_p_plot = Axes3D.scatter(ax3, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_pred)
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('p(x, y)_pred')

plt.show()

In [ ]:
fig4 = plt.figure(4)
ax4 = Axes3D(fig4)
real_p_plot = Axes3D.scatter(ax4, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_real)
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('p(x, y)_real')

plt.show()

In [ ]:
def real_derivatives(X, sigma_x, sigma_y):
    x1 = X[:, 0]
    x2 = X[:, 1]
    
    first_order_dx = x1/(sigma_x**2)
    first_order_dy = x2/(sigma_y**2)
    second_order_dy = 1/(sigma_y**2)
    
    return first_order_dx, first_order_dy, second_order_dy

dx, dy, ddy = real_derivatives(X_test, sigma_x, sigma_y)

In [ ]:
dx, dy, ddy = analytic_functions.real_derivatives(tModel.X_crossVal, sigma_x, sigma_y)

#for i in range(len(dx)):
 #   print('dx {}, dy {}, ddy {}'.format(dx[i], dy[i], ddy[i]))